# Primeiro passo importar todas as Libs que vamos utilizar


In [1]:
# Importa as opções do Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import Firefox
from time import sleep
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import FirefoxProfile
import json

# Definindo variáveis

In [2]:
# navegador...
f = Firefox()
# url da página inicial da academia das apostas
academia = "https://www.academiadasapostasbrasil.com/"
# abrindo o Navegador e Request
f.get(academia)
# waits 
wdw = WebDriverWait(f, 30, poll_frequency=1)

# DEFs que Pegam URLs e a guardam em um JSON

In [6]:
qt_analises_page = 1 # quantas vezes a lista de jogos foram att
def prox_urls(): 
    """
    Faz o click para listar os próximos jogos && tem um contador
    """
    global qt_analises_page
    next_btn = f.find_element(
        By.XPATH, 
        '//div[@class= "widget-double-container-left mb-content"]//span[contains(@class, "date-increment")]'
        )
    qt_analises_page += 1
    next_btn.click()  

def wait_expansao():
    """
    vai retornar TRUE até expandir tudo, ou seja enquanto houver jogos ocultos continua TRUE
    """
    try:
        btn_exp = f.find_element(By.XPATH, '//td[contains(@class, "footer")]')
    except:
        sleep(1)
        print('WARN: btn de expandir não existe mais')
        btn_exp = False
    
    return (True if bool(btn_exp) else False)
    
def click_expansao(): 
    """
     função q responsável pela expansão de todos os jogos
    """
    try:
        sleep(1)
        #while wait_expansao():
        if wait_expansao():
            btn_exp = f.find_element(By.XPATH, '//td[contains(@class, "footer")]')
            btn_exp.click()
            wdw.until_not(wait_expansao, "erro na def soup")     
    except:
        sleep(1)
        print('WARN: verificando se ainda tem btn_expandir para refazer o click')
        if wait_expansao():
            click_expansao()
    
urls = [] # vai guardar as urls
def soup():
    """
        cria objeto BS4;
        cria JSONs para cada dia, Ex: [ {hoje:[url, url]}, {17/07/20: [url, url]} ]
    """
    try:
        # tabela de jogos
        table_games = f.find_element(
            By.XPATH, 
            '//div[@class="widget-double livescores large"]').get_attribute(
                'outerHTML'
                )
        soup = BeautifulSoup(table_games, 'html.parser') # criando soup
        date = soup.find('label').attrs['data-displaydate'] # data
        href = soup.find_all('td', class_='score') # todas partidas
        print(f'DIA: {date} - Jogos adicionados: {len(href)}')
        all_urls = {date: [] }
        # preciso pegar a referencia da data p
        for link in href:
            all_urls[date].append(link.find('a').attrs['href'])
        urls.append(all_urls)
    except:
        print('soup error')

def gerar_json():
    with open('urls.json', 'w', encoding='utf-8') as jp:
        # criando arquivo JSON
        # json só faz dump ou load 
        # (o primeiro converte o obj em str, enquanto o load faz o oposto)
        jayzon = json.dumps(urls, indent=4)
        jp.write(jayzon)
    print('JSON gerado!')


# Criando JSON - processo de execução...

In [5]:
try:
    for i in range(3):
        print(f'iniciando {i+1}')
        if i > 0:
            prox_urls()
        wait_expansao()
        click_expansao()
        soup()
    print(f'Qt de dias analisados: {qt_analises_page}')
    gerar_json()
except:
    print('FATAL ERROR!!!!')

iniciando 1
WARN: verificando se ainda tem btn_expandir para refazer o click
WARN: verificando se ainda tem btn_expandir para refazer o click
WARN: btn de expandir não existe mais
DIA: Hoje - Jogos adicionados: 248
iniciando 2
WARN: btn de expandir não existe mais
WARN: verificando se ainda tem btn_expandir para refazer o click
WARN: verificando se ainda tem btn_expandir para refazer o click
WARN: btn de expandir não existe mais
DIA: Mon, 07/18/2022 - Jogos adicionados: 63
iniciando 3
WARN: btn de expandir não existe mais
WARN: verificando se ainda tem btn_expandir para refazer o click
WARN: verificando se ainda tem btn_expandir para refazer o click
WARN: btn de expandir não existe mais
DIA: Tue, 07/19/2022 - Jogos adicionados: 162
Qt de dias analisados: 3


Pegando Informações de cada JOGO

In [ ]:
gol_home_15 = []

url = ['https://www.academiadasapostasbrasil.com/stats/match/europa/uefa-nations-league/alemanha/italia/o1qZpnoJlQn5B']

wait_lt = WebDriverWait(f, 27, poll_frequency=1.5)

for u in url:
    def wait(f):
        click = f.find_element(By.XPATH, '//li[@data-id = "last_10g"][@market="teams_goals"]')
        click.click()
        last_ten = f.find_element(By.XPATH, '//li[@data-id = "last_10g"][@market="teams_goals"][contains(@class, "current")]')
        print('esperando click nos "10 ultimos" ')
        return bool(last_ten)
        
    f.get(u)

    # 10 ultimos jogos
    wait_lt.until(wait, "Não clicou nos '10 ultimos'")
    sleep(1)
    game = f.find_element(By.XPATH, '//table[@class="stat-goals"]/parent::td/parent::tr').get_attribute("outerHTML")
    soup = BeautifulSoup(game, 'html.parser')
    #gol_away_15.append(soup.find('td', class_='stats-wd-goalstime3 no-separator').text)
    gol_home_15.append(soup.find('td', class_='stats-wd-goalstime3 no-separator').text)

#f.quit()